![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## TF-IDF

Term Frequency Inverse Document Frequency (TF-IDF) jest po popularną i przydatną miarą unikalności danego terminu (słowa) dla danego dokumentu. Miara w istocie składa się z dwóch elementów, TF oraz IDF, które na końcu są ze sobą mnożone.

Term Frequency (TF), jest częstotliwością wystąpień terminu w dokumencie. Jako równanie zapisuje się go jak poniżej:

$$
TF = \frac{n_w}{n_d},
$$

gdzie: $n_w$ ile razy termin wystąpił w dokuemcie $n_d$ wszystkie słowa w dokumencie.

Z kolei Inverse Documents Frequency (IDF), zapisuje się jako:

$$
IDF = \log \left( \frac{c_d}{i_d} \right),
$$

gdzie: $c_d$ jest liczbą wszystkich dokumentów, $i_d$ jest liczbą dokumentów w których dane słowo wystąpiło. Należy pamiętać, że $\log$ jest logarytmem naturalnym przy podstawie $e$.

## Dane do analizy 

W przykładzie użyte zostaną dane z [Projektu Gutenberg](https://www.gutenberg.org). Poniższy skrypt pobierze dane jeżeli nie ma ich jeszcze w folderze `data`.

In [1]:
import os
import urllib.request

data_path = 'data/tf-idf_docs'
if not os.path.exists(data_path):
    os.mkdir(data_path)
    
book_links = {
    'grimms_fairy_tales': 'https://www.gutenberg.org/files/2591/2591-0.txt',
    'dracula': 'http://www.gutenberg.org/cache/epub/345/pg345.txt',
    'frankenstein': 'http://www.gutenberg.org/cache/epub/84/pg84.txt',
    'moby_dick': 'http://www.gutenberg.org/files/2701/2701-0.txt',
    'tom_sawyer': 'http://www.gutenberg.org/files/74/74-0.txt',
    'war_and_peace': 'http://www.gutenberg.org/files/2600/2600-0.txt'
}

for book, link in book_links.items():
    file_name = '{}.txt'.format(book)
    file_path = os.path.join(data_path, file_name)
    if not os.path.exists(file_path):
        print('Pobieranie pliku {} do {}'.format(link, file_path))
        urllib.request.urlretrieve(link, file_path)
        print('Pobieranie {} ukończone'.format(file_path))
    else:
        print('Plik {} jest już pobrany'.format(file_path))

Pobieranie pliku https://www.gutenberg.org/files/2591/2591-0.txt do data/tf-idf_docs/grimms_fairy_tales.txt
Pobieranie data/tf-idf_docs/grimms_fairy_tales.txt ukończone
Pobieranie pliku http://www.gutenberg.org/cache/epub/345/pg345.txt do data/tf-idf_docs/dracula.txt
Pobieranie data/tf-idf_docs/dracula.txt ukończone
Pobieranie pliku http://www.gutenberg.org/cache/epub/84/pg84.txt do data/tf-idf_docs/frankenstein.txt
Pobieranie data/tf-idf_docs/frankenstein.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/2701/2701-0.txt do data/tf-idf_docs/moby_dick.txt
Pobieranie data/tf-idf_docs/moby_dick.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/74/74-0.txt do data/tf-idf_docs/tom_sawyer.txt
Pobieranie data/tf-idf_docs/tom_sawyer.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/2600/2600-0.txt do data/tf-idf_docs/war_and_peace.txt
Pobieranie data/tf-idf_docs/war_and_peace.txt ukończone


## Zadanie 

Należy policzyć miary TF-IDF dla pobranych dokumentów. Kroki do wykoniania:

1. Odcztać dane, tak aby zachować informacje z której książki pochodzi dane zdanie.
1. Podzielić zdania (linie) na słowa; słowa powinny być sprowadzone do wspólnego znaku i nie posiadać interpunkcji.
1. Policzyć miarę TF.
1. Policzyć miarę IDF.
1. Złączyć wyniki tak, żeby wynikowa tabela miała kolumny: nazwa książki, słowo (termin), TF, IDF, TF-IDF.

In [2]:
import pyspark
import pyspark.sql.functions as func

spark = pyspark.sql.SparkSession.builder\
    .appName('tf-idf_sql')\
    .getOrCreate()

In [3]:
books = spark.read.text(data_path)

books.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|                                                                       |
|The Project Gutenberg EBook of War and Peace, by Leo Tolstoy           |
|                                                                       |
|This eBook is for the use of anyone anywhere at no cost and with almost|
|no restrictions whatsoever. You may copy it, give it away or re-use    |
|it under the terms of the Project Gutenberg License included with this |
|eBook or online at www.gutenberg.org                                   |
|                                                                       |
|                                                                       |
|Title: War and Peace                                                   |
|                                     

## Podpowiedzi

* Pliki mają nagłówki, ale można je zostawić i potraktować jako część dokumentu.
* Wszystko da się zrobić interfejsem Spark SQL (DataFrame lub zapytania SQL); nie ma potrzeby pisania dodatkowych funkcji.
* Wprawdzie wiemy jakie są książki, ale jest lepsza metoda dostania nazwy czytanego pliku w DataFrame; zobacz [manual Spark odnośnie funkcji](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). 
* Generalnie, w powyższym manualu jest sporo przydatnych funkcji.

In [9]:
lines = books.select(
    func.regexp_extract(func.input_file_name(), '\/(\w+)\.txt', 1).alias('book'),
    func.col('value').alias('line'))

lines.show(truncate=False)

+-------------+-----------------------------------------------------------------------+
|book         |line                                                                   |
+-------------+-----------------------------------------------------------------------+
|war_and_peace|                                                                       |
|war_and_peace|The Project Gutenberg EBook of War and Peace, by Leo Tolstoy           |
|war_and_peace|                                                                       |
|war_and_peace|This eBook is for the use of anyone anywhere at no cost and with almost|
|war_and_peace|no restrictions whatsoever. You may copy it, give it away or re-use    |
|war_and_peace|it under the terms of the Project Gutenberg License included with this |
|war_and_peace|eBook or online at www.gutenberg.org                                   |
|war_and_peace|                                                                       |
|war_and_peace|                 

In [19]:
words = lines.select('book',
            func.explode(func.split(func.lower(func.col('line')), '\W+')).alias('word'))\
    .where(func.length('word') > 0)
    
words.show()

+-------------+---------+
|         book|     word|
+-------------+---------+
|war_and_peace|      the|
|war_and_peace|  project|
|war_and_peace|gutenberg|
|war_and_peace|    ebook|
|war_and_peace|       of|
|war_and_peace|      war|
|war_and_peace|      and|
|war_and_peace|    peace|
|war_and_peace|       by|
|war_and_peace|      leo|
|war_and_peace|  tolstoy|
|war_and_peace|     this|
|war_and_peace|    ebook|
|war_and_peace|       is|
|war_and_peace|      for|
|war_and_peace|      the|
|war_and_peace|      use|
|war_and_peace|       of|
|war_and_peace|   anyone|
|war_and_peace| anywhere|
+-------------+---------+
only showing top 20 rows



In [20]:
words.groupby('book').count().show()

+------------------+------+
|              book| count|
+------------------+------+
|         moby_dick|222629|
|     war_and_peace|586914|
|           dracula|167267|
|        tom_sawyer| 77612|
|grimms_fairy_tales|105336|
|      frankenstein| 78475|
+------------------+------+



In [24]:
words.groupBy(func.length('word').alias('len')).count().orderBy('len').show(40)

+---+------+
|len| count|
+---+------+
|  1| 57996|
|  2|209996|
|  3|310464|
|  4|231122|
|  5|130029|
|  6| 98018|
|  7| 79385|
|  8| 52359|
|  9| 33195|
| 10| 19011|
| 11|  8461|
| 12|  4772|
| 13|  2302|
| 14|   755|
| 15|   245|
| 16|    93|
| 17|    24|
| 18|     5|
| 20|     1|
+---+------+



In [41]:
w = pyspark.sql.Window.partitionBy('book')

tf = words.groupBy('book', 'word')\
    .agg(func.count('*').alias('n_w'))\
    .withColumn('n_b', func.sum('n_w').over(w))\
    .withColumn('tf', func.col('n_w')/func.col('n_b'))\
    .orderBy(func.desc('tf'))
tf.show()

+------------------+----+-----+------+--------------------+
|              book|word|  n_w|   n_b|                  tf|
+------------------+----+-----+------+--------------------+
|grimms_fairy_tales| the| 7224|105336|  0.0685805422647528|
|         moby_dick| the|14718|222629| 0.06610998567122882|
|     war_and_peace| the|34725|586914|0.059165397315449966|
|      frankenstein| the| 4371| 78475|0.055699267282574065|
|grimms_fairy_tales| and| 5551|105336|0.052698032961190855|
|        tom_sawyer| the| 3971| 77612| 0.05116476833479359|
|           dracula| the| 8090|167267| 0.04836578643725301|
|        tom_sawyer| and| 3193| 77612| 0.04114054527650363|
|      frankenstein| and| 3046| 78475| 0.03881490920675374|
|     war_and_peace| and|22307|586914| 0.03800727193421864|
|      frankenstein|   i| 2849| 78475| 0.03630455559095253|
|           dracula| and| 5976|167267| 0.03572731022855674|
|      frankenstein|  of| 2760| 78475| 0.03517043644472762|
|         moby_dick|  of| 6743|222629|0.

In [52]:
n_d = words.select('book').distinct().count()

In [57]:
idf = words.groupBy('word')\
    .agg(func.countDistinct('book').alias('i_d'))\
    .withColumn('n_d', func.lit(n_d))\
    .withColumn('idf', func.log(func.col('n_d')/func.col('i_d')))
idf.show()

+-------------+---+---+------------------+
|         word|i_d|n_d|               idf|
+-------------+---+---+------------------+
|      embrace|  6|  6|               0.0|
|     tripping|  3|  6|0.6931471805599453|
|   circulates|  2|  6|1.0986122886681098|
|     incoming|  1|  6| 1.791759469228055|
|          fog|  5|  6|0.1823215567939546|
|   horsecloth|  1|  6| 1.791759469228055|
|       wields|  2|  6|1.0986122886681098|
|          few|  6|  6|               0.0|
|      flashed|  6|  6|               0.0|
|  freebooting|  1|  6| 1.791759469228055|
|  transaction|  3|  6|0.6931471805599453|
|    elocution|  1|  6| 1.791759469228055|
|    solemnity|  5|  6|0.1823215567939546|
| promontories|  1|  6| 1.791759469228055|
|    forgetful|  3|  6|0.6931471805599453|
|    arguments|  4|  6|0.4054651081081644|
|    recognize|  4|  6|0.4054651081081644|
| submissively|  1|  6| 1.791759469228055|
|gratification|  3|  6|0.6931471805599453|
|   roundabout|  3|  6|0.6931471805599453|
+----------

In [67]:
tf_idf = tf.join(idf, 'word')\
    .withColumn('tf_idf', func.col('tf')*func.col('idf'))\
    .select('book', 'word', 'tf', 'idf', 'tf_idf')
tf_idf.show()

+------------------+----+--------------------+---+------+
|              book|word|                  tf|idf|tf_idf|
+------------------+----+--------------------+---+------+
|grimms_fairy_tales| the|  0.0685805422647528|0.0|   0.0|
|         moby_dick| the| 0.06610998567122882|0.0|   0.0|
|     war_and_peace| the|0.059165397315449966|0.0|   0.0|
|      frankenstein| the|0.055699267282574065|0.0|   0.0|
|grimms_fairy_tales| and|0.052698032961190855|0.0|   0.0|
|        tom_sawyer| the| 0.05116476833479359|0.0|   0.0|
|           dracula| the| 0.04836578643725301|0.0|   0.0|
|        tom_sawyer| and| 0.04114054527650363|0.0|   0.0|
|      frankenstein| and| 0.03881490920675374|0.0|   0.0|
|     war_and_peace| and| 0.03800727193421864|0.0|   0.0|
|      frankenstein|   i| 0.03630455559095253|0.0|   0.0|
|           dracula| and| 0.03572731022855674|0.0|   0.0|
|      frankenstein|  of| 0.03517043644472762|0.0|   0.0|
|         moby_dick|  of|0.030288057710361183|0.0|   0.0|
|         moby

In [68]:
w = pyspark.sql.Window.partitionBy('book')\
    .orderBy(func.desc('tf_idf'))

tf_idf.withColumn('rank', func.rank().over(w))\
    .where('rank <= 3')\
    .drop('rank')\
    .show()

+------------------+-------+--------------------+------------------+--------------------+
|              book|   word|                  tf|               idf|              tf_idf|
+------------------+-------+--------------------+------------------+--------------------+
|         moby_dick|   ahab|0.002322249122980...| 1.791759469228055| 0.00416091185600665|
|         moby_dick|  whale|0.005587771584115277|0.6931471805599453|0.003873148119142...|
|         moby_dick| whales|0.001217271783999389| 1.791759469228055|0.002181058245605033|
|     war_and_peace| pierre|0.003344612668977...| 1.791759469228055|0.005992741420539759|
|     war_and_peace|   rost|0.001644193186736...| 1.791759469228055|0.002945998711574...|
|     war_and_peace|    sha| 0.00216897194478237|1.0986122886681098|0.002382859232314...|
|           dracula|helsing|0.001931044378149904| 1.791759469228055|0.003459967050049...|
|           dracula|   lucy|0.001775604273407...| 1.791759469228055|0.003181455770479128|
|         